In [1]:
# pip install opencv-python

In [1]:
import matplotlib
import sklearn
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import pathlib
import os
import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization

print("Versions of key libraries")
print("---")
print("tensorflow: ", tf.__version__)
print("numpy:      ", np.__version__)
print("matplotlib: ", matplotlib.__version__)
print("sklearn:    ", sklearn.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Versions of key libraries
---
tensorflow:  2.6.0
numpy:       1.22.3
matplotlib:  3.5.3
sklearn:     1.1.1
Num GPUs Available:  1


In [2]:
modelname   = 'Alexnet'

In [3]:
def createAlexNetModel():
    model = Sequential()
    model.add(Conv2D(filters=96, kernel_size=(11,11),strides=(4,4), input_shape=(227,227,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=(5,5),strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=384, kernel_size=(3,3),strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=384, kernel_size=(3,3),strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=(3,3),strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(4096, input_shape=(227*227*3,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Dense(4096, input_shape=(224*224*3,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Dense(1000))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Dense(4))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model       = createAlexNetModel() # This is meant for training
modelGo     = createAlexNetModel() # This is used for final testing

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0

In [4]:
                                                                                # Step 1
modelname       = modelname+"_"+str(datetime.datetime.now())[:-7].replace(' ','_').replace(":",'-')
folderpath      = 'models/'
filepath        = folderpath + modelname + ".hdf5"
checkpoint      = ModelCheckpoint(filepath, 
                                  monitor='val_accuracy', 
                                  verbose=0, 
                                  save_best_only=True, 
                                  mode='max')

csv_logger      = CSVLogger(folderpath+modelname +'.csv')                       # Step 2
callbacks_list  = [checkpoint,csv_logger]                                       # Step 3

print("Path to model:", filepath)
print("Path to log:  ", folderpath+modelname+'.csv')

Path to model: models/Alexnet_2022-09-16_15-07-04.hdf5
Path to log:   models/Alexnet_2022-09-16_15-07-04.csv


In [5]:
plotpath  = folderpath+modelname+'_plot.png'
plot_model(model, 
           to_file=plotpath, 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='TB')
print("Path to plot:", plotpath)

Path to plot: models/Alexnet_2022-09-16_15-07-04_plot.png


In [6]:
data_dir = pathlib.Path("dataset")
fish_images_dict = {
    'arowana': list(data_dir.glob('arowana/*.jpg')),
    'betta': list(data_dir.glob('betta/*.jpg')),
    'goldfish': list(data_dir.glob('goldfish/*.jpg')),
    'luohan': list(data_dir.glob('luohan/*.jpg'))
}
labelname = ['arowana','betta','goldfish','luohan']


In [7]:
X, y = [], []
for fish_name, images in fish_images_dict.items():
    for image in images:
        resized_img = tf.keras.preprocessing.image.load_img(
                image,
                grayscale=False,
                color_mode='rgb',
                target_size=(227,227),
                #keep_aspect_ratio=True,
                interpolation='nearest'
            )
        input_arr = tf.keras.preprocessing.image.img_to_array(resized_img)
        X.append(input_arr)
        y.append(labelname.index(fish_name))


C:\Anaconda3\envs\PRMLS\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [8]:
X = np.array(X)
oy = np.array(y)
y = to_categorical(oy)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=3000,test_size=1000, random_state=42)

In [9]:
X_train.shape
y_train.shape

(3000, 4)

In [10]:
model.fit(X_train,                            # Training data
          y_train,                            # Training label
          validation_data=(X_test, y_test),   # Validation data and label
          epochs=50,                       # The amount of epochs to be trained
          batch_size=32,
          shuffle=True,                     # To shuffle the training data
          callbacks=callbacks_list)         # Callbacks to execute the checkpoints

Epoch 1/50
94/94 [==============================] - 12s 78ms/step - loss: 0.8560 - accuracy: 0.7007 - val_loss: 52.2971 - val_accuracy: 0.1320
Epoch 2/50
94/94 [==============================] - 6s 60ms/step - loss: 0.6071 - accuracy: 0.7673 - val_loss: 0.8722 - val_accuracy: 0.7400
Epoch 3/50
94/94 [==============================] - 6s 61ms/step - loss: 0.5086 - accuracy: 0.8003 - val_loss: 1.2874 - val_accuracy: 0.7290
Epoch 4/50
94/94 [==============================] - 6s 60ms/step - loss: 0.4842 - accuracy: 0.8213 - val_loss: 0.5206 - val_accuracy: 0.8220
Epoch 5/50
94/94 [==============================] - 6s 60ms/step - loss: 0.4035 - accuracy: 0.8497 - val_loss: 0.9296 - val_accuracy: 0.6200
Epoch 6/50
94/94 [==============================] - 6s 60ms/step - loss: 0.3859 - accuracy: 0.8530 - val_loss: 0.9039 - val_accuracy: 0.7270
Epoch 7/50
94/94 [==============================] - 6s 60ms/step - loss: 0.3575 - accuracy: 0.8637 - val_loss: 0.4348 - val_accuracy: 0.8470
Epoch 8/50


In [11]:
                                                                                # Step 1
modelGo.load_weights(filepath)
modelGo.compile(loss='categorical_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy'])

predicts    = modelGo.predict(X_test)                                            # Step 2
print("Prediction completes.")

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [12]:
                                                                                # Step 1
                                                                                # Step 2
predout     = np.argmax(predicts,axis=1)
testout     = np.argmax(y_test,axis=1)

testScores  = metrics.accuracy_score(testout,predout)                           # Step 3

                                                                                # Step 4
print("Best accuracy (on testing dataset): %.2f%%" % (testScores*100))
print(metrics.classification_report(testout,
                                    predout,
                                    target_names=labelname,
                                    digits=4))

report = metrics.classification_report(testout,
                                    predout,
                                    target_names=labelname,
                                    digits=4,
                                      output_dict=True)

df = pd.DataFrame(report).transpose()
df.to_csv(folderpath+modelname+'_report.csv')

NameError: name 'predicts' is not defined